In [ ]:
!pip install langchain langchain_core langchain_community
!pip install faiss-cpu
!pip install sentence_transformers==2.2.2
!pip install InstructorEmbedding
!pip install langchain_groq
!pip install git+https://github.com/openai/whisper.git
!pip install langchain_core
!pip install langchain_community
!pip install python-dotenv
!pip install faiss-cpu
!pip install huggingface_hub
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2
!pip install torch
!pip install langchain_groq
!pip install transformers
!pip install torchaudio
!pip install pillow
import whisper
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.met

In [ ]:
#function to get text from the audio file
def audio_to_text(audio_path):
  model = whisper.load_model("base")
  result = model.transcribe(audio_path)
  return result["text"]


In [ ]:
#converting the raw text to chunks
def get_text_chunks(text):
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )
  chunks = text_splitter.split_text(text)
  return chunks

In [ ]:
#vector store to store embeddings
def get_vectorstore(text_chunks):
  embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
  vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
  return vectorstore

In [ ]:
#create conversational chain
def get_conversation_chain(vectorstore, groq_api_key):
  llm = ChatGroq(groq_api_key=groq_api_key, model_name = "llama3-70b-8192")
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
  conversation_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      retriever=vectorstore.as_retriever(),
      memory=memory
  )
  return conversation_chain

In [ ]:
def handle_userinput(conversation_chain, user_question):
    response = conversation_chain({'question': user_question})
    chat_history = response['chat_history']

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(f"User: {message.content}")
        else:
            print(f"Bot: {message.content}")

In [ ]:
def main():
  groq_api_key = "your key"
  print("Welcome to your audio assistant!!!!. Please upload your audio file.")

  audio_file = input("Enter the path to your audio file!!")
  raw_text = audio_to_text(audio_file)
  text_chunks = get_text_chunks(raw_text)
  vectorstore = get_vectorstore(text_chunks)
  conversation_chain = get_conversation_chain(vectorstore, groq_api_key)

  if vectorstore:
            # Create conversation chain
            conversation_chain = get_conversation_chain(vectorstore, groq_api_key)

            while True:
                user_question = input("Ask something (type 'exit' to quit): ")
                if user_question.lower() in ['exit' , 'quit']:
                    break
                handle_userinput(conversation_chain, user_question)


if __name__ == '__main__':
  main()

Welcome to your audio assistant!!!!. Please upload your audio file.


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 105MiB/s]
/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


User: summarize the speech in bullet points
Bot: Here is a summary of the speech in bullet points:

**Personal Story**

* The speaker (Barack Obama) grew up overseas and had to learn American education through extra lessons with his mother
* His father left when he was two, and he was raised by a single mom who struggled to provide
* He got into trouble, but got second chances and went on to college and law school

**Importance of Education**

* Education is key to success in any career
* You can't drop out of school and expect to get a good job
* Education is not just important for personal success, but also for the future of the country

**Overcoming Challenges**

* Everyone faces challenges, but it's not an excuse to give up
* You can't let your circumstances define you; you have to work hard to overcome them
* The story of America is about people who kept going and tried harder, even when things got tough

**Goals and Perseverance**

* Set goals for your education and do everything